## 3.17
スパムデータについて、表3.3と同様の解析(最小二乗法、最良部分集合、リッジ回帰、lasso、主成分回帰、部分最小二乗法をした時の係数と評価誤差を示す、また予測誤差が最小となるパラメータを10分割交差確認により決定)をせよ


In [69]:
import pandas as pd
import numpy as np

### dataset
>4601通のメール全てについて、email(0), spam(1)のどちらかのラベル
メール本文中に最も頻出する57種類の単語と句読点記号の相対出現頻度も与えられている

In [33]:
dataset = pd.read_csv("./data/chapter3/spambase.data", sep=",", header=None)

In [34]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [35]:
X = dataset.loc[:, 0:56]
y = dataset[57]

In [85]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

### 最小二乗法

In [91]:
liner_reg = linear_model.LinearRegression()
skf = StratifiedKFold(n_splits=10, shuffle=True)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    liner_reg.fit(X_train, y_train)
    print(liner_reg.score(X_test, y_test))
    
liner_reg.coef_

0.54962642194
0.465251537445
0.552913282087
0.497011064148
0.580486350674
0.534095863124
0.532261338275
0.558131081918
0.441107043001
0.557543646606


array([ -6.04822033e-02,  -1.18744742e-02,   3.81040657e-02,
         1.17453750e-02,   8.48616267e-02,   1.24739435e-01,
         2.05222758e-01,   1.07382953e-01,   6.78090407e-02,
         1.68086751e-02,   5.72498035e-02,  -2.66458072e-02,
         1.81404073e-02,  -9.53544421e-04,   1.04642840e-02,
         7.27383782e-02,   5.06984567e-02,   5.52375330e-02,
         1.44663403e-02,   6.05950371e-02,   5.39777404e-02,
         4.55116983e-02,   1.68105027e-01,   8.74716436e-02,
        -2.24153675e-02,  -2.28626944e-02,  -1.22538058e-02,
         7.74631616e-03,  -7.80032708e-03,  -4.96029605e-02,
        -2.41330188e-02,  -4.91463867e-02,  -4.17462414e-02,
         1.14811721e-01,  -3.06781938e-02,   5.91294251e-03,
        -3.34049647e-02,  -5.10296801e-02,  -1.64811802e-02,
         4.34154982e-02,  -1.07345015e-02,  -3.71194292e-02,
        -5.84234878e-02,  -3.04043540e-02,  -3.38285995e-02,
        -3.68018216e-02,  -1.90384155e-01,  -5.65823326e-02,
        -1.50005478e-01,

### lasso

In [87]:
lasso_cv = linear_model.LassoCV()
skf = StratifiedKFold(n_splits=10, shuffle=True)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    lasso_cv.fit(X_train, y_train)
    print(lasso_cv.alpha_, lasso_cv.score(X_test, y_test))
    
lasso_cv.coef_

0.074663803356 0.261479292568
0.0723400727555 0.239790976694
0.0738132835655 0.292641796174
0.0719681369377 -0.563192742156
0.0738876532908 0.273553784982
0.0778220288559 0.255862834378
0.0748604740726 0.249324065553
0.0734043415627 0.28589964262
0.0739763929746 0.227163377349
0.0714284556851 0.299266535206


array([ 0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        0.02514984,  0.        ,  0.        ,  0.02737165,  0.        ,
        0.07387063,  0.        ,  0.        ,  0.        , -0.0306168 ,
       -0.        , -0.01150504, -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.00081846,
        0.00019454,  0.00014677])

### ridge

In [84]:
ridge_cv = linear_model.RidgeCV()
skf = StratifiedKFold(n_splits=10, shuffle=True)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    ridge_cv.fit(X_train, y_train)
    print(ridge_cv.alpha_, ridge_cv.score(X_test, y_test))
    
ridge_cv.coef_

10.0 0.427556073439
10.0 0.549070821765
10.0 0.494522245649
10.0 0.547305208917
10.0 0.568063325936
10.0 0.516702979968
10.0 0.544445327352
10.0 0.577551914307
10.0 0.576346412536
10.0 0.490830090746


array([ -6.90426085e-02,  -1.19262686e-02,   4.19173552e-02,
         1.21509773e-02,   8.61342443e-02,   1.08941789e-01,
         2.19709792e-01,   8.81796492e-02,   8.00723766e-02,
         1.47837383e-02,   5.59995487e-02,  -2.57541752e-02,
         2.06273969e-02,  -4.30194600e-03,   2.10434281e-02,
         8.92535491e-02,   5.37038479e-02,   5.88105247e-02,
         1.33056910e-02,   6.25344387e-02,   5.10653327e-02,
         4.45124204e-02,   1.75052804e-01,   8.94186113e-02,
        -2.24132730e-02,  -2.33546901e-02,  -1.18656795e-02,
         1.80211308e-03,  -4.11909358e-03,  -4.86147038e-02,
        -1.97981094e-02,   2.28716820e-02,  -5.19061511e-02,
         2.36368532e-02,  -2.94285388e-02,   2.02297055e-02,
        -3.29216176e-02,  -5.83331746e-02,  -1.77345916e-02,
         4.62662654e-02,  -7.69942304e-03,  -3.81199980e-02,
        -5.58852413e-02,  -3.42418105e-02,  -3.37934301e-02,
        -3.76957832e-02,  -1.35948050e-01,  -6.49190264e-02,
        -1.31707171e-01,

### PCA